<a href="https://colab.research.google.com/github/edenmui/edenmui.github.io/blob/main/CMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title CMI Compiler { form-width: "30%", display-mode: "form" }
#@markdown 

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
ss = gc.open_by_key('1Fm8hLCjZvhAJ2gnlglzj6hclzSCgNF1YTMLXOJlAvnk')
ws = ss.worksheet('CMI')

import os
path = os.getcwd() #@param {type:"string"}
 
colab_html_as_string = "\u003Col class=\"sc-pXZzD bjbVGr\">\u003Cli class=\"adult\">\u003Cdiv class=\"task-wrapper\">\u003Cspan class=\"title\">MIDEA\u003C/span>\u003Cspan class=\"subtitle\">raymond_wei,eden_mui\u003C/span>\u003C/div>\u003C/li>\u003Cli class=\"adult\">\u003Cdiv class=\"task-wrapper\">\u003Cspan class=\"title\">XINFN\u003C/span>\u003Cspan class=\"subtitle\">raymond_wei,eden_mui\u003C/span>\u003C/div>\u003C/li>\u003Cli class=\"adult\">\u003Cdiv class=\"task-wrapper\">\u003Cspan class=\"title\">LINYA\u003C/span>\u003Cspan class=\"subtitle\">raymond_wei,eden_mui\u003C/span>\u003C/div>\u003C/li>\u003C/ol>" #@param {type: "string"}
from lxml import html
 
import pandas as pd

with open('colab.html', 'w') as ref:
    ref.write(colab_html_as_string)
 
files = [path + '/' + f for f in os.listdir(path) if ('CMI' in f) and (f.endswith('xls'))]
 
for f in files:
    global sup_prof, sup_sup
    if os.path.getsize(files[0]) > os.path.getsize(files[1]):
        sup_prof    = files[0]
        sup_sup     = files[1]
    else:
        sup_prof    = files[1]
        sup_sup     = files[0]
pd.options.display.max_columns          = None
pd.options.display.max_rows             = None
df_supplementary                        = pd.DataFrame(pd.read_excel(sup_sup, header=8))
df_profile                              = pd.DataFrame(pd.read_excel(sup_prof,header=8))

df_profile.set_index('Supplier code')
df = pd.merge(
    df_profile,
    df_supplementary,
    how='inner',
    left_on='Supplier code',
    right_on='Code'
    ).filter(
    items=[
    'Supplier code',
    'Supplier name 1',
    'Supplier name 2',
    'Trading',
    'Product type',
    'Country code',
    'Adress',
    'e mail',
    'Contacts'
    ]
)
df = df.convert_dtypes(infer_objects=True,convert_string=True,convert_boolean=False,convert_integer=False)
df = df.drop_duplicates(subset='Supplier code',keep='first')
df['Supplier Name'] = (
    df['Supplier name 1'].astype(str) + 
    ' ' + 
    df['Supplier name 2'].astype(str)
).str.replace(
    ' nan',
    ''
).str.replace(
    "<NA>",
    ""
).str.strip()
df['Country']       = df['Country code']
df['Sector']        = df['Product type'].astype(str).str.extract('(^[0-9]{1})')
df['First Name']    = None
df['Last Name']     = None
df['BCC']           = None
dfnew = df.filter(
    items=[
        'Supplier code',
        'Supplier Name',
        'Trading',
        'Sector',
        'Product type',
        'Country',
        'Adress',
        'e mail',
        'First Name',
        'Last Name',
        'Contacts'
    ]
)
dfnew['Product type']   = dfnew['Product type'].astype(str).str.replace(" \| ",",")
dfnew['Trading']        = dfnew['Trading'].astype(str).str.replace('\|',',')

from bs4 import BeautifulSoup as bs
lst = {}
with open('colab.html', 'r') as ref:
    soup = bs(ref,"html.parser")
    s_code = [code.get_text() for code in soup.find_all('span',class_='title')]
    print(s_code)
    colabs = [team.get_text() for team in soup.find_all('span',class_='subtitle')]
    print(colabs)
    data = {'Supplier code':s_code, 'BCC':colabs}
    df2 = pd.DataFrame.from_dict(data)

df = df2.copy(deep=True)
 
merged = pd.merge(
    dfnew,
    df,
    how='left',
    on='Supplier code'
).filter(
    items=[
        'Supplier code',
        'Supplier Name',
        'Trading',
        'Sector',
        'Product type',
        'Country',
        'Adress',
        'e mail',
        'First Name',
        'Last Name',
        'BCC',
        'Contacts']
).set_index(
    'Supplier code')
 
def l2str(list_):
    str_new = ","
    return str_new.join(list_)
merged['BCC'] = merged['BCC'].apply(lambda x:str(x).split(',')).apply(lambda x: list(name + '@carrefour.com' for name in x)).apply(lambda x:l2str(x))
merged['BCC'] = merged['BCC'].apply(lambda x: x + ', gsasupplier_contract@carrefour.com')
 
blacklist = {
    'AD':True,
    'AG':True,'AI':True,'AW':True,'BH':True,'BM':True,'BQ':True,'BS':True,'BW':True,'BZ':True,'CK':True,'CY':True,'DM':True,'ET':True,'GD':True,'GG':True,'GH':True,'GI':True,'IM':True,'IR':True,'JE':True,'KN':True,'KY':True,'LC':True,'LI':True,'LR':True,'MC':True,'MH':True,'MS':True,'MT':True,'NR':True,'NU':True,'PA':True,'RS':True,'SC':True,'SM':True,'SY':True,'TC':True,'TT':True,'VC':True,'VG':True,'VI':True,'VU':True,'WS':True,'YE':True,
}
rex = {
    'AF':True,'AM':True,'AO':True,'BD':True,'BF':True,'BI':True,'BJ':True,'BO':True,'BT':True,'CD':True,'CF':True,'CG':True,'CK':True,'CV':True,'DJ':True,'ER':True,'ET':True,'FM':True,'GN':True,'GQ':True,'GW':True,'HT':True,'ID':True,'IN':True,'KE':True,'KG':True,'KH':True,'KI':True,'KM':True,'LA':True,'LK':True,'LR':True,'LS':True,'MG':True,'ML':True,'MM':True,'MN':True,'MR':True,'MW':True,'MZ':True,'NE':True,'NG':True,'NP':True,'NR':True,'NU':True,'PH':True,'PK':True,'RW':True,'SB':True,'SD':True,'SL':True,'SN':True,'SO':True,'SS':True,'ST':True,'SY':True,'TD':True,'TG':True,'TJ':True,'TL':True,'TO':True,'TV':True,'TZ':True,'UG':True,'UZ':True,'VN':True,'VU':True,'WS':True,'YE':True,'ZM':True,
}
merged['blacklist'] = merged['Country'].apply(lambda x:True if x in list(blacklist.keys()) else False)
merged['REX'] = merged['Country'].apply(lambda x:True if x in list(rex.keys()) else False)

ws.clear()
from gspread_dataframe import set_with_dataframe as swf
from gspread_dataframe import get_as_dataframe as gaf
swf(ws, merged, include_index=True)


ws = ss.worksheet('SupplierList')
df = gaf(ws).dropna(axis=0, how='all').dropna(axis=1, how='all')

df = df.append(merged)
df['Supplier code'] = df['Supplier code'].astype(str)
df.drop_duplicates(subset='Supplier code', inplace=True, keep='last')
swf(ws, df)


print("""useful link:\n
Google Sheet: https://docs.google.com/spreadsheets/d/1Fm8hLCjZvhAJ2gnlglzj6hclzSCgNF1YTMLXOJlAvnk/edit#gid=0\n
Contract delivery agent: https://supplierperformance.carrefour.com/SupplierContract/rest/contract/page \n 

""")

In [ ]:
ws = ss.worksheet('SupplierList')
df = gaf(ws).dropna(axis=0, how='all').dropna(axis=1, how='all')
print(df.shape)
df = df.append(merged)
print(df.shape)

In [ ]:

df.drop_duplicates(subset='Supplier code', inplace=True, keep='last')
print(df.shape)
ws.clear()
swf(ws, df)
print(df.shape)

In [ ]:
blacklist = {
    'AD':True,
    'AG':True,'AI':True,'AW':True,'BH':True,'BM':True,'BQ':True,'BS':True,'BW':True,'BZ':True,'CK':True,'CY':True,'DM':True,'ET':True,'GD':True,'GG':True,'GH':True,'GI':True,'IM':True,'IR':True,'JE':True,'KN':True,'KY':True,'LC':True,'LI':True,'LR':True,'MC':True,'MH':True,'MS':True,'MT':True,'NR':True,'NU':True,'PA':True,'RS':True,'SC':True,'SM':True,'SY':True,'TC':True,'TT':True,'VC':True,'VG':True,'VI':True,'VU':True,'WS':True,'YE':True,
}
rex = {
    'AF':True,'AM':True,'AO':True,'BD':True,'BF':True,'BI':True,'BJ':True,'BO':True,'BT':True,'CD':True,'CF':True,'CG':True,'CK':True,'CV':True,'DJ':True,'ER':True,'ET':True,'FM':True,'GN':True,'GQ':True,'GW':True,'HT':True,'ID':True,'IN':True,'KE':True,'KG':True,'KH':True,'KI':True,'KM':True,'LA':True,'LK':True,'LR':True,'LS':True,'MG':True,'ML':True,'MM':True,'MN':True,'MR':True,'MW':True,'MZ':True,'NE':True,'NG':True,'NP':True,'NR':True,'NU':True,'PH':True,'PK':True,'RW':True,'SB':True,'SD':True,'SL':True,'SN':True,'SO':True,'SS':True,'ST':True,'SY':True,'TD':True,'TG':True,'TJ':True,'TL':True,'TO':True,'TV':True,'TZ':True,'UG':True,'UZ':True,'VN':True,'VU':True,'WS':True,'YE':True,'ZM':True,
}

str(rex.keys())

In [ ]:
string = "'AF', 'AM', 'AO', 'BD', 'BF', 'BI', 'BJ', 'BO', 'BT', 'CD', 'CF', 'CG', 'CK', 'CV', 'DJ', 'ER', 'ET', 'FM', 'GN', 'GQ', 'GW', 'HT', 'ID', 'IN', 'KE', 'KG', 'KH', 'KI', 'KM', 'LA', 'LK', 'LR', 'LS', 'MG', 'ML', 'MM', 'MN', 'MR', 'MW', 'MZ', 'NE', 'NG', 'NP', 'NR', 'NU', 'PH', 'PK', 'RW', 'SB', 'SD', 'SL', 'SN', 'SO', 'SS', 'ST', 'SY', 'TD', 'TG', 'TJ', 'TL', 'TO', 'TV', 'TZ', 'UG', 'UZ', 'VN', 'VU', 'WS', 'YE', 'ZM'"
string.replace("'","""
""").replace(",","").replace(" \n","")


In [ ]:
list(rex.keys())

";".join(f"{n}" for n in list(rex.keys()))

In [ ]:
string = """
['AF',
 'AM',
 'AO',
 'BD',
 'BF',
 'BI',
 'BJ',
 'BO',
 'BT',
 'CD',
 'CF',
 'CG',
 'CK',
 'CV',
 'DJ',
 'ER',
 'ET',
 'FM',
 'GN',
 'GQ',
 'GW',
 'HT',
 'ID',
 'IN',
 'KE',
 'KG',
 'KH',
 'KI',
 'KM',
 'LA',
 'LK',
 'LR',
 'LS',
 'MG',
 'ML',
 'MM',
 'MN',
 'MR',
 'MW',
 'MZ',
 'NE',
 'NG',
 'NP',
 'NR',
 'NU',
 'PH',
 'PK',
 'RW',
 'SB',
 'SD',
 'SL',
 'SN',
 'SO',
 'SS',
 'ST',
 'SY',
 'TD',
 'TG',
 'TJ',
 'TL',
 'TO',
 'TV',
 'TZ',
 'UG',
 'UZ',
 'VN',
 'VU',
 'WS',
 'YE',
 'ZM']
 """
 